# Accessing German Historical Newspapers using the OpenAi GPT4 Model and "in-context Learning"
## Example: Event Detection

*Notebook created by Sarah Oberbichler (oberbichler@ieg-mainz.de)*

This notebook shows how LLMs can be used to support research with historical newspapers. In this example, the OpenAi GPT-4o model is used to detect events types and event arguments within a corpus on German historical newsapers with the help of "in-context learning". In-context learning means that the models gets provided with examples (or even one example) to address a new task. For furhter information se, for example, [here](https://https://www.lakera.ai/blog/what-is-in-context-learning).

Events in historical newspapers are characterized by interactions between entities (such as countries, organizations, or individuals) that deviate from
typical interaction patterns. Detecting those events (locating them), and extracting event arguments (type of event, who, where, when, etc.) helps to provide corpora for furhter analyis.


### 1.   Import your dataset for the event detection

Import a dataset that contains the prepared articles for the event detection (ideally with OCR-post correction). In this example, I am using a small dataset containing articles from the German Newspaper Portal ([Deutsches Zeitungsportal](https://https://www.deutsche-digitale-bibliothek.de/newspaper)). An instruction on how to extract newspaper pages or issues via an API can be found [here](https://https://deepnote.com/app/karl-kragelin-b83c/Zeitungsportal-API-d9224dda-8e26-4b35-a6d7-40e9507b1151).

In [1]:
import pandas as pd
excel_file_path = 'Erdbeben.xlsx'

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)

# Display the DataFrame
df.head()

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext,context,article,article_corrected
0,22NYUNGOY64IXJJMTMPKYR7UADDF6GVH-ALTO8578354_D...,3,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1851-05-17 12:00:00,"['Köln', 'Kleve (Kreis Kleve)', 'Jülich']",['ger'],d9797b92-931d-4a4b-b99d-21a041a30d1d,['/data/altos/22/NY/22NYUNGOY64IXJJMTMPKYR7UAD...,ALTO8578354_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,piemontesisch = schweizerischen Telegraphen = ...,fallenen Unweiters . — Die mininisterielle „ Z...,"Eine frühere Citadelle , vermuthlich das Caste...","Eine frühere Citadelle, vermutlich das Castell..."
1,24I6NCIJKY6FXQ6WLXRZ642KPYXATVNO-ALTO2890815_D...,2,Neuwiedische Nachrichten,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2731244-6,1851-07-01 12:00:00,['Neuwied'],['ger'],480ccbcd-b912-40fe-aa7f-60c2b1c3fb31,['/data/altos/24/I6/24I6NCIJKY6FXQ6WLXRZ642KPY...,ALTO2890815_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Stegen Haussuchung gehalten , und zwar , wie e...",ätig . Namentlich hat sie sich in den jüngsten...,Am 2 . April gegen 6½ Uhr Morgens wurde Valpa ...,Am 2. April gegen 6½ Uhr Morgens wurde Valpara...
2,26ZR5I7GWW3DAFUXMVRLMLBY3PJJHO6X-FILE_0003_DDB...,3,Staats- und gelehrte Zeitung des Hamburgischen...,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Car...,2260914-3,1851-10-23 12:00:00,['Hamburg'],['ger'],0459305c-9a65-429b-8ccf-c36487f6d94c,['/data/altos/26/ZR/26ZR5I7GWW3DAFUXMVRLMLBY3P...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,»HP h* dis ^iatS- »iFcrä ’t'l'llfö sst'sspf UH...,"ber. Der Sü-ele, der von Allem, was Villault a...",Zn port of Spain (Jamaika) war wie der ein lei...,"""In Port of Spain (Jamaika) war wie der ein le..."
3,27PZTJBLDL6AHEAFKPBUTAMHVVSGDVSM-FILE_0003_DDB...,3,Dresdner Journal und Anzeiger,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2889118-1,1851-05-05 12:00:00,['Dresden'],['ger'],075be8b6-4850-4b88-876e-0725946cf523,['/data/altos/27/PZ/27PZTJBLDL6AHEAFKPBUTAMHVV...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,991 worden sind. Der preußische General Brandt...,"sein wir-, daß Alles, waS nicht gerade entschi...","Palermo, 14. April. Ein Erdbeben hat in Messin...","Palermo, 14. April. Ein Erdbeben hat in Messin..."
4,2AFZAFNOG2CCXPTK2O4O4UG6H7USN2OD-FILE_0005_DDB...,5,Schwäbischer Merkur : mit Schwäbischer Kronik ...,VNHXUCEEKHOUSYH4NVOUBHJGSRMOGK7J,Württembergische Landesbibliothek,2751625-8,1851-09-06 12:00:00,['Stuttgart'],['ger'],ddff4a11-d067-45f2-b15b-dff431bb4d25,['/data/altos/2A/FZ/2AFZAFNOG2CCXPTK2O4O4UG6H7...,FILE_0005_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"1207 auswärtigen Angelegenheiten, hat seine En...","1207 auswärtigen Angelegenheiten, hat seine En...","Ueber daS Erdbeben, welches am 14Aug. in de: n...","Über das Erdbeben, welches am 14. August in de..."


## Setting up the requirements for the OPENAI GPT4o model

GPT-4 is a large language model (LLM) that processes both images and text as input and generates text as output. It employs an architecture based on the Transformer model, which consists of multiple layers of stacked decoders. These decoders utilize various neural networks and incorporate the attention mechanism to effectively handle and generate language.

In [2]:
!pip install OpenAI
from openai import OpenAI

In [3]:
# @markdown ##### Get an API key at https://platform.openai.com/api-keys, activate the billing, save your key as .env file. To do so, take following steps:
# @markdown - Open a Notepad and write OPENAI_API_KEY = "your key"
# @markdown - Click on Save option and change the file type to 'All files'
# @markdown - Keep the file name as .env.
# @markdown - Hit Save. Now the file is an .env file.
# @markdown - Load the .env file into the drive/MyDrive
!pip install python-dotenv

import os
import dotenv

#Set the REPLICATE_API_TOKEN environment variable
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# @markdown Load the .env file into the drive/MyDrive
dotenv.load_dotenv('/content/drive/MyDrive/.env')

client = OpenAI(
  api_key=os.getenv('OPENAI_API_KEY'),  # this is also the default, it can be omitted
)

# Run model for Event Detection

To run Event_Detection, it is essential to formulate a precise prompt. For example, it needs to be specified what event arguments should be extracted. Providing examples further helps to let the model learn. A guide on how to write effective prompts can be found also [here](https://https://support.google.com/a/users/answer/14200040?hl=en).

Depending on the size of the dataframe, it can take a while to load.

In [5]:
def events(article_corrected):
    # Define the prompt to instruct OpenAI to separate events
    prompt = f"Wenn ein Event vorliegt: Bitte teile die Events ein in: Ereignistyp, Ereignisdatum, Ereignisorte, beteiligte Personen. Wenn kein Event vorliegt, schreibe 'Kein Event'  \n\n{article_corrected}\n\n---\n"
    system = f"Du bist Experte in Event Detection. Verwende \n\n{example}\n\n als Model. Kommentier selbst nicht"
    # Call the OpenAI completion endpoint
    client = OpenAI()

    response = client.chat.completions.create(
        model="gpt-4o-2024-05-13",
        messages=[
          {
            "role": "system",
            "content": system
          },
          {
            "role": "user",
            "content": prompt
          }
        ],
        max_tokens=3000,
        temperature=0
    )

    # Extract separated events from the response
    events = response.choices[0].message.content.strip().split("\n")

    # Return the separated events
    return events

# Apply the 'events' function to each article in the dataframe
example = ['Ereignistyp: Erdbeben', 'Ereignisdatum: 16. Dezember', 'Ereignisort: Neapel und Sizilien, Italien', 'Beteiligte Personen: Pfarrer Mancini, einige königliche Gendarmen, 40 gerettete Pfarrgenossen, 155 verstorbene Pfarrgenossen']
df['events'] = df['article_corrected'].apply(events)

# Print the modified dataframe
df

,page_id,pagenumber,paper_title,provider_ddb_id,provider,zdb_id,publication_date,place_of_distribution,language,thumbnail,pagefulltext,pagename,preview_reference,plainpagefulltext,context,article,article_corrected,events
0,22NYUNGOY64IXJJMTMPKYR7UADDF6GVH-ALTO8578354_D...,3,Kölnische Zeitung. 1803-1945,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2719361-5,1851-05-17 12:00:00,"['Köln', 'Kleve (Kreis Kleve)', 'Jülich']",['ger'],d9797b92-931d-4a4b-b99d-21a041a30d1d,['/data/altos/22/NY/22NYUNGOY64IXJJMTMPKYR7UAD...,ALTO8578354_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,piemontesisch = schweizerischen Telegraphen = ...,fallenen Unweiters . — Die mininisterielle „ Z...,"Eine frühere Citadelle , vermuthlich das Caste...","Eine frühere Citadelle, vermutlich das Castell...","[Ereignistyp: Erdbeben , Ereignisdatum: 1. Ja..."
1,24I6NCIJKY6FXQ6WLXRZ642KPYXATVNO-ALTO2890815_D...,2,Neuwiedische Nachrichten,VKNQFFAKOR4XZWJJKUX3NGYSZ3QZAXCW,Universitäts- und Landesbibliothek der Rheinis...,2731244-6,1851-07-01 12:00:00,['Neuwied'],['ger'],480ccbcd-b912-40fe-aa7f-60c2b1c3fb31,['/data/altos/24/I6/24I6NCIJKY6FXQ6WLXRZ642KPY...,ALTO2890815_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"Stegen Haussuchung gehalten , und zwar , wie e...",ätig . Namentlich hat sie sich in den jüngsten...,Am 2 . April gegen 6½ Uhr Morgens wurde Valpa ...,Am 2. April gegen 6½ Uhr Morgens wurde Valpara...,"[Ereignistyp: Erdbeben , Ereignisdatum: 2. Ap..."
2,26ZR5I7GWW3DAFUXMVRLMLBY3PJJHO6X-FILE_0003_DDB...,3,Staats- und gelehrte Zeitung des Hamburgischen...,BZVTR553HLJBDMQD5NCJ6YKP3HMBQRF4,Staats- und Universitätsbibliothek Hamburg Car...,2260914-3,1851-10-23 12:00:00,['Hamburg'],['ger'],0459305c-9a65-429b-8ccf-c36487f6d94c,['/data/altos/26/ZR/26ZR5I7GWW3DAFUXMVRLMLBY3P...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,»HP h* dis ^iatS- »iFcrä ’t'l'llfö sst'sspf UH...,"ber. Der Sü-ele, der von Allem, was Villault a...",Zn port of Spain (Jamaika) war wie der ein lei...,"""In Port of Spain (Jamaika) war wie der ein le...",[Kein Event]
3,27PZTJBLDL6AHEAFKPBUTAMHVVSGDVSM-FILE_0003_DDB...,3,Dresdner Journal und Anzeiger,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,2889118-1,1851-05-05 12:00:00,['Dresden'],['ger'],075be8b6-4850-4b88-876e-0725946cf523,['/data/altos/27/PZ/27PZTJBLDL6AHEAFKPBUTAMHVV...,FILE_0003_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,991 worden sind. Der preußische General Brandt...,"sein wir-, daß Alles, waS nicht gerade entschi...","Palermo, 14. April. Ein Erdbeben hat in Messin...","Palermo, 14. April. Ein Erdbeben hat in Messin...","[Ereignistyp: Erdbeben , Ereignisdatum: 14. A..."
4,2AFZAFNOG2CCXPTK2O4O4UG6H7USN2OD-FILE_0005_DDB...,5,Schwäbischer Merkur : mit Schwäbischer Kronik ...,VNHXUCEEKHOUSYH4NVOUBHJGSRMOGK7J,Württembergische Landesbibliothek,2751625-8,1851-09-06 12:00:00,['Stuttgart'],['ger'],ddff4a11-d067-45f2-b15b-dff431bb4d25,['/data/altos/2A/FZ/2AFZAFNOG2CCXPTK2O4O4UG6H7...,FILE_0005_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"1207 auswärtigen Angelegenheiten, hat seine En...","1207 auswärtigen Angelegenheiten, hat seine En...","Ueber daS Erdbeben, welches am 14Aug. in de: n...","Über das Erdbeben, welches am 14. August in de...","[Ereignistyp: Erdbeben , Ereignisdatum: 14. A..."
5,2EHKK4YK2LPQMJ5E4LK43KG6UKKVNXLL-FILE_0002_DDB...,2,Intelligenz- und Wochenblatt für Frankenberg m...,265BI7NE7QBS4NQMZCCGIVLFR73OCOSL,Sächsische Landesbibliothek - Staats- und Univ...,3107190-9,1851-04-30 12:00:00,"['Frankenberg/Sa.', 'Sachsenburg (Frankenberg,...",['ger'],40525de4-83d3-453b-80bb-cf2fc759892e,['/data/altos/2E/HK/2EHKK4YK2LPQMJ5E4LK43KG6UK...,FILE_0002_DDB_FULLTEXT,https://api.deutsche-digitale-bibliothek.de/bi...,"auch Alles das erlogen ist, was man Son der Mi...","auch Alles das erlogen ist, was man Son der Mi...",Ferner schöpft der alte WetterfabrikaKt seins ...,Komme

In [6]:
df.to_excel('event_erdbeben.xlsx', index=False)